In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots
import random as rd # generating random numbers
import datetime # manipulating date formats

from tqdm import tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")
!pwd

In [ ]:
sales=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
sales

In [ ]:
# you can use this tip to save some memory : #downcast data to save memory : from 134.4+ MB, we went to 61.6+ MB
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

print("-- avant")
print(sales.info())
sales = downcast_dtypes(sales)
print("-- apres")
print(sales.info())

In [ ]:
# nombre de produits par shops
# unique_products = []
# for shop_id in tqdm(sales.shop_id):
#     unique_products.append({'shop_id': shop_id,
#                             'n_products' : len(sales[sales.shop_id == shop_id].item_id.unique())  }
#                           )
    
# unique_products = pd.DataFrame(unique_products)
# unique_products

In [ ]:
# monthly_sales=sales.groupby(["date_block_num","shop_id","item_id"])[
#     "date","item_price","item_cnt_day"].agg({"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

# monthly_sales

In [ ]:
# monthly_sales.reset_index(drop= True, inplace = True)


In [ ]:
# monthly_sales


In [ ]:
# from dateutil.parser import parse
# sales['rdate'] = sales['date'].apply(lambda d: parse(d))

In [ ]:
# sorted(sales['rdate'].unique())

In [ ]:
len(sales['rdate'].unique())

In [ ]:
len(sales.date_block_num.unique())

In [ ]:
item_id = 32
shop_id = 0

cond = (sales.item_id == item_id) & (sales.shop_id == shop_id)
sales[cond]


In [ ]:
len(sales.shop_id.unique())

In [ ]:
sales.shape

In [ ]:
import dabl
sales_frac = sales.sample(frac = 0.1)
sales_clean = dabl.clean(sales_frac, verbose = 1)

In [ ]:
dabl.plot(sales_frac, target_col = 'item_cnt_day')

In [ ]:
ts=sales.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,8))
plt.title('Total Sales of the company')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts);

In [ ]:
plt.plot(ts.rolling(window=12,center=False).mean(),label='Rolling Mean');

In [ ]:
from statsmodels.tsa.seasonal import STL
res = STL(ts, robust = True, period = 12).fit() 
fig, ax = plt.subplots(5,1, figsize = (18,12))
plt.subplot(5,1,1)
plt.plot(ts.values)
plt.plot(res.trend)

plt.subplot(5,1,2)
plt.plot(res.trend)
plt.subplot(5,1,3)
plt.plot(res.seasonal)
plt.subplot(5,1,4)
plt.plot(ts.values - res.seasonal)
plt.subplot(5,1,5)
plt.plot(res.resid)

plt.grid()

In [ ]:
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    
test_stationarity(ts)

# TS des produits les plus répandus

pour chaque produit 

- voir les dates min / max 
- le nombre de dates avec des ventes
- par jour
- aggreger par semaine, par mois






In [ ]:

sales=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")

sales.head()


In [ ]:
from dateutil.parser import parse
sales['rdate'] = sales['date'].apply(lambda d: parse(d))

In [ ]:
df = sales.groupby(by = 'item_id').agg({'rdate': ["min","max"]}).reset_index()

In [ ]:
df['days'] =  df.apply(lambda d : (d['rdate']['max'] - d['rdate']['min']).days   , axis = 1   )

df.head()

In [ ]:
def njours(item_id):
    return len(sales[ (sales.item_id == item_id)   ].rdate.unique())

df['njours'] =  df.item_id.apply(lambda id : njours(id) )
df.head()

In [ ]:
df.sort_values(by = ['njours', 'days'], ascending = [False, False],  inplace = True)
df.head()

In [ ]:
df.sample(5)

In [ ]:
len(sales[ (sales.item_id == 5822)   ].rdate.unique())

In [ ]:
item_id = 5822
# item_id = 12552

In [ ]:
df2 = sales[sales.item_id == item_id][['item_cnt_day', 'rdate']].groupby( by = [ 'rdate']  ).sum().reset_index()
df2.head()

In [ ]:
ts = pd.Series(index = df2.rdate, data = df2.item_cnt_day.values)
ts.plot(figsize = (18,6))

# Prophet

see https://facebook.github.io/prophet/docs/quick_start.html#python-api

In [ ]:
from fbprophet import Prophet

In [ ]:
ts = df2[['rdate','item_cnt_day']].copy()
ts.rename(columns = {'rdate': 'ds', 'item_cnt_day': 'y'  }, inplace = True)
ts


In [ ]:
m = Prophet()
m.fit(ts)

In [ ]:
future = m.make_future_dataframe(periods=30)
future.tail(30)

In [ ]:
# ?m.make_future_dataframe

In [ ]:

forecast = m.predict(future)


In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30)

In [ ]:

fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

# Log

In [ ]:
ts = df2[['rdate','item_cnt_day']].copy()
ts.rename(columns = {'rdate': 'ds', 'item_cnt_day': 'y'  }, inplace = True)
ts['y'] = np.log(ts['y'] + 1)
ts

# ts = pd.Series(index = df2.rdate, data = np.log(df2.item_cnt_day.values))
# ts.plot(figsize = (18,6))


In [ ]:
m = Prophet()
m.fit(ts)
future = m.make_future_dataframe(periods=30)

forecast = m.predict(future)

fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)